In [ ]:
from plot_statistics import *

In [ ]:
# experiment_name = 'Random Search'
# experiment_name = 'F-DENSER'
# experiment_name = 'Stepper-Decay'
experiment_name = 'Stepper-Adaptive'

stats = load_stats(experiment_name)
print_statistics(stats, experiment_name)
do_all_plots(stats, experiment_name, plot_best_run=True, group_pictures=True)

In [ ]:
experiment_name = 'Random Search'
stats = load_stats( experiment_name)
print_statistics(stats, experiment_name)

experiment_name = 'F-DENSER'
stats = load_stats(experiment_name)
print_statistics(stats, experiment_name)

experiment_name = 'Stepper-decay'
stats = load_stats(experiment_name)
print_statistics(stats, experiment_name)

experiment_name = 'Stepper-adaptive'
stats = load_stats(experiment_name)
print_statistics(stats, experiment_name)

In [ ]:
experiment_name = 'Stepper-Adaptive'
stats = load_stats(experiment_name)
print_statistics(stats, experiment_name)

do_all_plots(stats, experiment_name, plot_best_run=True, group_pictures=True)